<a href="https://colab.research.google.com/github/Hrishkesh/Invoice-/blob/master/Invoice_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install --user tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 76kB/s 
     |████████████████████████████████| 3.8MB 39.2MB/s 
     |████████████████████████████████| 450kB 41.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3cadb3f206a4a00216d8db684883c9198ea09e7f0bcac7742d45c4a6da37b239
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [75]:
!pip install --upgrade wandb
!wandb login 47ae077cee2c514d2b004ca904e0a365185a06c1

     |████████████████████████████████| 1.4MB 3.2MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
     |████████████████████████████████| 460kB 15.6MB/s 
     |████████████████████████████████| 102kB 8.0MB/s 
     |████████████████████████████████| 112kB 14.5MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=f2e000efbedadb9f820a7f003b19dc1e133d6f55a83b20a8ce7f3dee47a3c730
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=a909bd4bd2f704cb3e5610713ef0d1fca8601e2804e34e9d4bd95edfa20dac14
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=3662

In [0]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [18]:
!wget https://www.dropbox.com/s/u2dccpjgs3ynm8x/invoice_eng.tar.gz

--2020-05-23 20:38:36--  https://www.dropbox.com/s/u2dccpjgs3ynm8x/invoice_eng.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/u2dccpjgs3ynm8x/invoice_eng.tar.gz [following]
--2020-05-23 20:38:36--  https://www.dropbox.com/s/raw/u2dccpjgs3ynm8x/invoice_eng.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4e4c9cfbe1c1a355d9f10e3404.dl.dropboxusercontent.com/cd/0/inline/A4SWQaY38ANTmPRlvii1Pc8EZt0z0MlBWdGo3NrDTTiO9bHZSJGzOEUwM8-tVDEDJWq5uNL03MA1MO68csMqHzC08jnjUF0IbC4E2ygBg0cvixsD7oleVSRHmTpnkEh2f7U/file# [following]
--2020-05-23 20:38:36--  https://uc4e4c9cfbe1c1a355d9f10e3404.dl.dropboxusercontent.com/cd/0/inline/A4SWQaY38ANTmPRlvii1Pc8EZt0z0MlBWdGo3NrDTTiO9bHZSJGzOEUwM8-tVDEDJWq5uNL03MA1MO68csMqHzC08jnjUF0I

In [22]:
!ls

drive  invoice_eng.tar.gz  sample_data


In [0]:
!gunzip invoice_eng.tar.gz

In [24]:
!ls

drive  invoice_eng.tar	sample_data


In [25]:
!tar -xvf invoice_eng.tar

invoice_eng/
invoice_eng/outdir_np_chargrid_1h/
invoice_eng/outdir_np_chargrid_1h/X51006555806.npy
invoice_eng/outdir_np_chargrid_1h/X51007339152.npy
invoice_eng/outdir_np_chargrid_1h/X51007339126.npy
invoice_eng/outdir_np_chargrid_1h/X51006556656.npy
invoice_eng/outdir_np_chargrid_1h/X51006619563.npy
invoice_eng/outdir_np_chargrid_1h/X51007579726.npy
invoice_eng/outdir_np_chargrid_1h/X51005757290.npy
invoice_eng/outdir_np_chargrid_1h/X51006913031.npy
invoice_eng/outdir_np_chargrid_1h/X51008142032.npy
invoice_eng/outdir_np_chargrid_1h/X51007339139.npy
invoice_eng/outdir_np_chargrid_1h/X51005568914.npy
invoice_eng/outdir_np_chargrid_1h/X51007846307.npy
invoice_eng/outdir_np_chargrid_1h/X51007846305.npy
invoice_eng/outdir_np_chargrid_1h/X51005568898.npy
invoice_eng/outdir_np_chargrid_1h/X51007103675.npy
invoice_eng/outdir_np_chargrid_1h/X51005746140.npy
invoice_eng/outdir_np_chargrid_1h/X51005676541.npy
invoice_eng/outdir_np_chargrid_1h/X51006620189.npy
invoice_eng/outdir_np_chargrid_1h/

In [30]:
!ls invoice_eng/

'Colab Notebooks'	       outdir_np_bbox_anchor_mask   outdir_np_gt_1h
 outdir_np_bbox_anchor_coord   outdir_np_chargrid_1h	    output


In [84]:
!git clone https://github.com/Hrishkesh/Invoice-.git

Cloning into 'Invoice-'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 16 (delta 4), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [33]:
!ls Invoice-/

Invoice_new.ipynb  network.py  preprocessing.py  README.md


In [65]:
import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password
os.environ['GITHUB_USER'] = user
os.environ['GITHUB_PASSWORD'] = user

GitHub user··········
GitHub password··········


In [0]:
!git config --global user.email "hrishkesh.pattepu@gmail.com"
!git config --global user.name "Hrishkesh"

In [0]:
!cd Invoice- && git remote set-url origin https://$GITHUB_AUTH@github.com/$GITHUB_USER/Invoice-.git

In [87]:
!cd Invoice- && git pull

Already up to date.


In [0]:
!cd Invoice- && git add .

In [102]:
!cd Invoice- && git commit -m "training"

[master e6d83c3] training
 1 file changed, 1 insertion(+), 1 deletion(-)


In [103]:
!cd Invoice- && git push

Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 290 bytes | 290.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Hrishkesh/Invoice-.git
   3114e0b..e6d83c3  master -> master


In [104]:
!python Invoice-/network.py

wandb: Tracking run with wandb version 0.8.36
wandb: Run data is saved locally in wandb/run-20200523_212728-30al4jnc
wandb: Syncing run pretty-surf-5
wandb: ⭐️ View project at https://app.wandb.ai/hrishkeshpattepu/invoice
wandb: 🚀 View run at https://app.wandb.ai/hrishkeshpattepu/invoice/runs/30al4jnc
wandb: Run `wandb off` to turn off syncing.

2020-05-23 21:27:29.561925: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-05-23 21:27:29.567421: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-05-23 21:27:29.567791: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x24d0a00 executing computations on platform Host. Devices:
2020-05-23 21:27:29.567977: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incar